## Obtaining the Best Chi-Square Cut

This iPython notebook will help you ascertain the best chi-square cut for a certain supernova's ATLAS light curve. After running a cell, the descriptions located above that cell will help you interpret the plots and make decisions about the supernova.

This notebook takes into account ATLAS's periodic replacement of the difference image reference templates, which may cause step discontinuities in flux. Two template changes have been recorded at MJDs 58417 and 58882. More information can be found here: https://fallingstar-data.com/forcedphot/faq/.

In [1]:
# import modules, set preliminary variables, etc.

import numpy as np
import pandas as pd
import sys, copy, requests, time, json
from pdastro import pdastrostatsclass, AandB, AnotB, AorB, not_AandB

# for getting discovery date from TNS
from collections import OrderedDict
from astropy.time import Time

# plotting
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pylab as matlib
import warnings
warnings.simplefilter('error', RuntimeWarning)
warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = [12, 7]
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times'

# ATLAS template changes
global tchange1
global tchange2
tchange1 = 58417
tchange2 = 58882

global dir
global lc_info

lc_info = {}

In [2]:
# Enter the target SN name:
tnsname = '2019vxm'

# Enter your data directory (where the SN light curves are located):
dir = '/Users/sofiarest/Google Drive/My Drive/College/STScI Research Paper/atlaslc_chisquare/brightsne/'

# Optionally, enter the SN's discovery date (if None is entered, it will be 
# fetched automatically from TNS):
discdate = None

In [3]:
# get discovery date if needed, load in light curve, account for template changes, 
# and add uJy/duJy column

def get_tns_data(tnsname):
	try:
		get_obj = [("objname",tnsname), ("objid",""), ("photometry","1"), ("spectra","1")]
		get_url = 'https://www.wis-tns.org/api/get/object'
		json_file = OrderedDict(get_obj)
		get_data = {'api_key':'2eca323a16b17d78fbc99cd6f1f801699a81a91c','data':json.dumps(json_file)}
		response = requests.post(get_url, data=get_data, headers={'User-Agent':'tns_marker{"tns_id":104739,"type": "bot", "name":"Name and Redshift Retriever"}'})
		json_data = json.loads(response.text,object_pairs_hook=OrderedDict)
		return json_data
	except Exception as e:
		return 'Error: \n'+str(e)

def get_discdate(tnsname):
	json_data = get_tns_data(tnsname)
	discoverydate = json_data['data']['reply']['discoverydate']
	date = list(discoverydate.partition(' '))[0]
	time = list(discoverydate.partition(' '))[2]
	disc_date_format = date+'T'+time
	dateobjects = Time(disc_date_format, format='isot', scale='utc')
	disc_date = dateobjects.mjd
	return disc_date

def load_lc(lc_info):
	lc_info['lc'] = pdastrostatsclass()
	filename = dir+lc_info['tnsname']+'/'+lc_info['tnsname']+'_i000.o.lc.txt'
	try:
		print('Loading SN %s light curve at %s...' % (lc_info['tnsname'], filename))
		lc_info['lc'].load_spacesep(filename,delim_whitespace=True)
	except Exception as e:
		print('Could not load light curve for SN %s at %s: %s' % (lc_info['tnsname'], filename, str(e)))
		sys.exit()
	
	# get median baseline flux and subtract from all measurements to account for template shifts
	baseline = np.where(lc_info['lc'].t['MJD'] < lc_info['mjdstart'])[0]
	median_bflux = np.median(lc_info['lc'].t.loc[baseline,'uJy'])
	print('Subtracting median flux %0.1f uJy from light curve flux to account for template shifts...' % median_bflux)
	lc_info['lc'].t['uJy'] -= median_bflux

	# add flux/dflux column
	print('Adding uJy/duJy column to light curve...')
	lc_info['lc'].t['uJy/duJy'] = lc_info['lc'].t['uJy']/lc_info['lc'].t['duJy']
	lc_info['lc'].t = lc_info['lc'].t.replace([np.inf, -np.inf], np.nan)


lc_info['tnsname'] = tnsname

if discdate is None:
	print('Obtaining SN %s discovery date from TNS...' % lc_info['tnsname'])
	discdate = get_discdate(lc_info['tnsname'])
	print('Discovery date: ',discdate)
lc_info['mjdstart'] = discdate

load_lc(lc_info)

Obtaining SN 2019vxm discovery date from TNS...
Discovery date:  58818.04
Loading SN 2019vxm light curve at /Users/sofiarest/Google Drive/My Drive/College/STScI Research Paper/atlaslc_chisquare/brightsne/2019vxm/2019vxm_i000.o.lc.txt...
Subtracting median flux 30.0 from light curve to account for template shifts...
Adding uJy/duJy column to light curve...
